In [4]:
import os

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split

import matplotlib.pyplot as plt

import pytorch_lightning as pl

from skimage import io

In [12]:
DATA_DIR = 'data\Abstract_gallery'

In [20]:
class AbstractGalleryDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len([image for image in os.listdir(self.root_dir) if os.path.isfile(os.path.join(self.root_dir, image))])

    def __getitem__(self, index):
        image_path = os.listdir(self.root_dir)[index]
        image = io.imread(image_path)

        if self.transform:
            image = self.transform

        return image